In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.utils import to_categorical
import numpy as np

import pandas as pd
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import itertools


In [9]:

#workplace = "WFH"
workplace = "CICIMA"
#workplace = "colaboratory"

#patterns and samples

if workplace == "WFH": 
    #transflectance averages
    transflectance_data_path = r"E:\CICIMA\2023_Q4_TRANSMITTANCE_AND_TRANSFLECTANCE\2023_Q4_TRANSMITTANCE_AND_TRANSFLECTANCE\2023NOV_TRANSFLECTANCE\ASC\new_names\average"
    transmittance_data_path = r"E:\CICIMA\2023_Q4_TRANSMITTANCE_AND_TRANSFLECTANCE\2023_Q4_TRANSMITTANCE_AND_TRANSFLECTANCE\2023NOV_TRANSMITTANCE\ASC\new_name\average"
    #reports
    root_path = r"E:\CICIMA\Estudio Optico Escarabajos"
    #code name samples path
    code_name_path = r"E:\CICIMA\Estudio Optico Escarabajos\CODE NAME SAMPLES 2024.txt"
    
if workplace == "CICIMA":
    #transflectance averages
    transflectance_data_path = r"C:\Users\EstebanSoto\Downloads\VINICIO\VINICIO\2023-11-TRANSFLECTANCE_AND_TRANSMITTANCE\new names\transflectance\Modificado\ASC\average"
    transmittance_data_path = r"C:\Users\EstebanSoto\Downloads\VINICIO\VINICIO\2023-11-TRANSFLECTANCE_AND_TRANSMITTANCE\new names\transmittance\Modificado\ASC\average"
    #reports
    root_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos"
    #code name samples path
    code_name_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\CODE NAME SAMPLES 2024.txt"
    


#report path
current_date = datetime.now().date()

save_location = os.path.join(root_path, "data_analysis", "machine_learning",f"{current_date}" )
report_location = os.path.join(save_location, "report")


#absorptance and reflectance save path
save_path = os.path.join(save_location , r"keras_test_model")



In [10]:
#Create folders 
folders_to_create = [save_location, report_location, save_path]
for folder in folders_to_create:
    try:
        os.makedirs(folder)
    except Exception as e: 
        print(e)

In [11]:
def get_metadata_and_dataframe(file_location):
     #definitions
    #Logic to read ASCII data
    import os
    import pandas as pd
    import re
    
    def get_sample_code_from_filename(row_str, file_location):
        print("string")
        print(file_location)
        filename = os.path.basename(file_location)
        re1 = r"([a-zA-Z\d]+)(?:-\d)*(?:.Sample)*.(?:txt)*(?:ASC)*"
        #Names are in the form CODE-MEASUREMENTNUMBER.TXT
        p = re.compile(re1)
        m = p.match(filename)
        print(f"match filename: {m}")
        if m:
            print(f"group 1: {m.group(1)}")
            return(m.group(1))
        return get_sample_code(file_str)

    def get_sample_code(row_str):
        #Tries to get the sample code from the file, if it does not match
        #it tries to get it from the filename. 
        print("string")
        print(row_str)
        re1 = r"([a-zA-Z\d]+)(?:-\d)*(?:.Sample)*.(?:txt)*(?:ASC)*"
        #Names are in the form CODE-MEASUREMENTNUMBER.TXT
        p = re.compile(re1)
        m = p.match(row_str)
        print(f"match: {m}")
        if m:
            return(m.group(1))
        else: 
            ""

    def responses(str):
        re1 = "\d+/(\d+,\d+) \d+,\d+/(\d+,\d+)"
        p = re.compile(re1)
        m= p.match(str)
        if m:
            return m.group(1),m.group(2)    
        else:
            return "",""
    def attenuator_settings(str):
        re1 = "S:(\d+,\d+) R:(\d+,\d+)"
        p = re.compile(re1)
        m= p.match(str)
        if m:
            return m.group(1),m.group(2)    
        else:
            return "",""
    def slit_pmt_aperture(str):
        re1 = "\d+/servo \d+,\d+/(\d+,\d+)"
        p = re.compile(re1)
        m= p.match(str)
        if m:
            return m.group(1)    
        else:
            return ""
    #Initializa metadata dict
    metadata = {}
    
    #Read header
    lines = []
    with open(file_location) as myfile:
        lines = myfile.readlines()[0:90] 
    metadata["header"] = "".join(lines)
    
    
    #read_metadata
    f = open(file_location)
    
    df = pd.DataFrame()
    with f as data_file:
        for index, row in enumerate(data_file): #0-89

            row_str = row.strip()
            if index +1 == 3: #Filename and extension
                metadata["filename"]= row_str
                metadata["code"] = get_sample_code_from_filename(row_str, file_location)
            if index + 1 == 4: #date DD/MM/YYYY
                metadata["date"]= row_str
            if index + 1 == 5:#Time HH:MM:SS.SS
                metadata["time"]= row_str
            if index + 1 == 8:#user
                metadata["user"]= row_str
            if index + 1 == 9:#description
                metadata["description"]= row_str
            if index + 1 == 10:#minimum wavelength
                metadata["minimum_wavelength"]= row_str
            if index + 1 == 12:#equipment name
                metadata["equipment"]= row_str
            if index + 1 == 13:#equipment series
                metadata["series"]= row_str
            if index + 1 == 14:#data visualizer version, equipment version, date and time
                metadata["software"]= row_str
            if index + 1 == 21:#Operating mode
                metadata["operating_mode"]= row_str
            if index + 1 == 22: #Number of cycles
                metadata["cycles"]= row_str
            if index + 1 == 32: #range/servo
                metadata["slit_pmt"]= slit_pmt_aperture(row_str)
            if index + 1 == 33:
                metadata["response_ingaas"], metadata["response_pmt"]= responses(row_str)
            if index + 1 == 35: #pmt gain, if 0 is automatic
                metadata["pmt_gain"]= row_str
            if index + 1 == 36: #InGaAs detector gain
                metadata["ingaas_gain"]= row_str
            if index + 1 == 42:#monochromator wavelength nm
                metadata["monochromator_change"]= row_str
            if index + 1 == 43:#lamp change wavelength
                metadata["lamp_change"]= row_str
            if index + 1 == 44:#pmt wavelength
                metadata["pmt_change"]= row_str
            if index + 1 == 45:#beam selector
                metadata["beam_selector"]= row_str
            if index + 1 == 46:
                metadata["cbm"]= row_str
            if index + 1 == 47: #cbd status, on/off
                metadata["cbd_status"]= row_str
            if index + 1 == 48: #attenuator percentage
                metadata["attenuator_sample"], metadata["attenuator_reference"]= attenuator_settings(row_str)
            if index + 1 == 49:
                metadata["polarizer"]= row_str
            if index + 1 == 80:
                metadata["units"]= row_str
            if index + 1 == 81:
                metadata["measuring_mode"]= row_str
            if index + 1 == 84:
                metadata["maximum_wavelength"]= row_str
            if index + 1 == 85:
                metadata["step"]= row_str
            if index + 1 == 86:
                metadata["number_of_datapoints"]= row_str
            if index + 1 == 88:
                metadata["maximum_measurement"]= row_str
            if index + 1 == 89:
                metadata["minimum_measurement"]= row_str
            if index +1 == 90:
                break
        df = pd.read_csv(f, sep="\t", decimal =".", names=["wavelength", metadata["measuring_mode"]]).dropna()
        df["wavelength"],df[metadata["measuring_mode"]] = df["wavelength"].astype(float), df[metadata["measuring_mode"]].astype(float)
        return metadata, df

In [12]:
#Create folder if not exists
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
        
def filter_substring_elements(path_strings, substring):
    filtered_paths = [path for path in path_strings if substring in path]
    return filtered_paths

In [ ]:

# Example data preparation
# X_train, y_train, X_val, y_val = ...

# Convert species labels to numerical format
# y_train = to_categorical(y_train)
# y_val = to_categorical(y_val)

# Define model architecture
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(num_wavelengths, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
# model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate model
# loss, accuracy = model.evaluate(X_val, y_val)

# Make predictions
# predictions = model.predict(X_test)